In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(0)

# 데이터셋 읽기

In [ ]:
# 데이터파일 읽기
data = pd.read_csv('data/cs-training.csv')
data.shape

In [ ]:
data.head()

In [ ]:
data.columns[0]

In [ ]:
# 첫 컬럼의 index 항목 제거 (Unnamed: 0)
data.drop(columns=data.columns[0], inplace=True) #inplace=True: 원본(data)를 변경

In [ ]:
data.head(2)

- 제공되는 데이터셋의 컬럼명이 이해하기 어렵거나 사용하기 어렵다면 변경하도록 한다.
    - 컬럼명을 소문자로 변경함.
    - ex) 약어를 원래 의미의 단어들로 변경. 영문->한글 등등

In [ ]:
 [col.lower() for col in data.columns]

In [ ]:
data.columns = [col.lower() for col in data.columns]  #data.columns = 컬럼리스트 : 전체컬럼명을 다 변경할 경우 사용.
data.head(2)

# EDA

In [ ]:
data.info()

In [ ]:
data.describe().T  #dataframe/ndarray .T (Transpose-전처) : 컬럼<->행

In [ ]:
# 타겟(seriousdlqin2yrs) 분포 확인
data['seriousdlqin2yrs'].value_counts()

In [ ]:
data['seriousdlqin2yrs'].value_counts(normalize=True)

In [ ]:
# 원소의 개수
len(data['seriousdlqin2yrs']), data['seriousdlqin2yrs'].size, data['seriousdlqin2yrs'].shape

In [ ]:
# 비율로 보기
data['seriousdlqin2yrs'].value_counts()/len(data['seriousdlqin2yrs'])
# 불균형 데이터 - 평가지표: accuaracy(X). recall, precision, f1, roc auc

In [ ]:
plt.figure(figsize=(7,6))
sns.countplot(x='seriousdlqin2yrs', data=data)
plt.show()

# 결측치 처리

## 결측치 확인

In [ ]:
# data.isnull()
data.isna().sum() # True개수

In [ ]:
data.isna().mean() # True의 비율

In [ ]:
y = data.isna().mean()
plt.figure(figsize=(8,5))
plt.title('컬럼별 결측치 비율')
sns.barplot(x=y, y=data.columns)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 중앙값, 평균값 확인
data.aggregate(['median', 'mean'])[['monthlyincome','numberofdependents']].T

## 결측치 처리

### monthly income 확인

In [ ]:
np.round(data.monthlyincome.describe(), 2)  #np.round(값, 2) 소숫점 2자리 이하에서 반올림

In [ ]:
data.monthlyincome.hist(bins=300)
plt.xlim(0, 50000)
plt.show()

In [ ]:
data.monthlyincome.median()

In [ ]:
# fillna(채울값) 결측치를 채울값으로 채운다.
# monthly income 의 결측치 중앙값으로 대체
data.monthlyincome.fillna(data.monthlyincome.median(), inplace=True)

### numberofdependents

In [ ]:
data.numberofdependents.value_counts().sort_index() #sort_index() 행이름(index)으로 정렬

In [ ]:
data.numberofdependents.value_counts()/len(data)

In [ ]:
# 최빈값이 0인데 비율이 58% 정도 됨. 결측치 최빈값인 0 으로 대체
data.numberofdependents.fillna(0, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data.numberofdependents.describe()

## 이상치 확인 및 처리

### IQR 기반 이상치 검출
- IQR : 3분위수-1분위수
- 이상치 기준 (rate는 일반적으로 1.5사용)
    - 극단적으로 작은 값 범위
         - 1분위수 - IQR*rate 보다 작은수
    - 극단적으로 큰 값 범위
        - 3분위 + IQR*rate 보다 큰수

In [ ]:
def get_outlier(data, rate=1.5):
    """
    IQR 기반으로 Outlier 값 조회 메소드
    [Parameter]
        data: array-like -Outlier 계산할 데이터
        rate: IQR에 몇배를 극단치 계산에 사용할 지 비율. rate를 크게하면 outlier범위를 넓게 잡는다. 작게 주면 범위를 좁게 잡는다.
    [Return]
        bool type ndarray: 각 원소별 outlier 여부 (True: Outlier(이상치), False: 정상범위값)
    """
    q1 = np.quantile(data, q=0.25)  #분위수 계산: np.quantile(값, q='분위')
    q3 = np.quantile(data, q=0.75)
    IQR = q3 - q1
    return (data < q1 - IQR * rate) | (data > q3 + IQR * rate)

In [ ]:
get_outlier(np.array([1, 100,10,2,3,-100]))

## 각 컬럼별 이상치 처리

### revolvingutilizationofunsecuredlines
- 전체 운용가능한 돈 대비 현재 운용가능한 돈의 비율 (남은신용한도+통장잔고/ 총신용한도+통장잔고)
- 1초과하는 값들을 1로 변경한다.

In [ ]:
#확인
print(data['revolvingutilizationofunsecuredlines'].describe())
print('1 초과값 개수',(data['revolvingutilizationofunsecuredlines'] > 1).sum())

In [ ]:
data.loc[data['revolvingutilizationofunsecuredlines']>1, 'revolvingutilizationofunsecuredlines']

In [ ]:
data.loc[data['revolvingutilizationofunsecuredlines']>1, 'revolvingutilizationofunsecuredlines'] = 1

In [ ]:
#확인
print(data['revolvingutilizationofunsecuredlines'].describe())
print('1 초과값 총개수',(data['revolvingutilizationofunsecuredlines'] > 1).sum())

### age
- 대출자 나이
- 최소값이 0, 최대값 109
- 중위수로 변환 

In [ ]:
data['age'].describe()

In [ ]:
# 0이 한명, 그다음은 21부터 존재 한다. 
data['age'].value_counts().sort_index()

In [ ]:
data.loc[data['age']==0]

In [ ]:
data.loc[data['age']==0, 'age'] = data['age'].median()

In [ ]:
data['age'].describe()

### numberoftime30-59dayspastduenotworse, numberoftime60-89dayspastduenotworse, numberoftimes90dayslate

- 30 ~ 59, 60 ~ 89, 90이상 연체한 횟수
- 96, 98 두개의 값을 가지는 행이 있다.
    - 96은 5개 98은 264개로 세 컬럼의 같은 행이 같이 두 값을 가지고 있다. 
    - 이런 경우 특정 의미를 표현하는 코드 값일 수 있다. 그래서 제거하지 않고 유지한다.

In [ ]:
data['numberoftime30-59dayspastduenotworse'].describe()

In [ ]:
data['numberoftime30-59dayspastduenotworse'].value_counts().sort_index()

### numberoftime60-89dayspastduenotworse

In [ ]:
data['numberoftime60-89dayspastduenotworse'].describe()

In [ ]:
data['numberoftime60-89dayspastduenotworse'].value_counts().sort_index()

### numberoftimes90dayslate

In [ ]:
data['numberoftimes90dayslate'].describe()

In [ ]:
data['numberoftimes90dayslate'].value_counts().sort_index()

In [ ]:
data.loc[(data['numberoftime30-59dayspastduenotworse'] > 95) & 
         (data['numberoftime60-89dayspastduenotworse'] > 95) & 
         (data['numberoftimes90dayslate'] > 95)].shape

# 위 세개 컬럼에서 96, 98 을 가지는 행이 동일하다.
# 96, 98 은 은행에서 사용하는 코드값으로 추측할 수 있다. 

In [ ]:
data.loc[(data['numberoftime30-59dayspastduenotworse'] == 96) & 
         (data['numberoftime60-89dayspastduenotworse'] == 96) & 
         (data['numberoftimes90dayslate'] == 96), ['numberoftime30-59dayspastduenotworse','numberoftime60-89dayspastduenotworse','numberoftimes90dayslate']]

### debtratio 
- 소득 대비 부채비율(대출상환금+생활비/소득)
- 이상치가 아닌 값들 중 최대값으로 대체한다. 

In [ ]:
data['debtratio'].describe()

In [ ]:
get_outlier(data['debtratio']).sum() #이상치 개수

In [ ]:
get_outlier(data['debtratio']).mean()

In [ ]:
# 9분위수
np.quantile(data['debtratio'], q=0.9)

In [ ]:
# outlier 아닌 값들 중 최대값으로 대체
# ~ : not
max_value_debt = np.max(data.debtratio[~get_outlier(data['debtratio'])]) # 정상범위 값 중 최대값 조회
data.loc[data['debtratio']>max_value_debt, 'debtratio'] = max_value_debt

In [ ]:
max_value_debt

In [ ]:
get_outlier(data['debtratio']).sum()

In [ ]:
data['debtratio'].describe()

In [ ]:
data['debtratio'].hist(bins=30)

## monthlyincome
- 월간 소득
- 이상치를 이상치 아닌 값들의 최대 값으로 대체한다.

In [ ]:
get_outlier(data['monthlyincome']).sum()

In [ ]:
get_outlier(data['monthlyincome']).mean()

In [ ]:
np.round(data.monthlyincome.describe(), 3)
# max: 3,000,000 

In [ ]:
data.monthlyincome.hist(bins=500)
plt.xlim(0,10000)
plt.show()

In [ ]:
max_value_income = np.max(data.monthlyincome[~get_outlier(data['monthlyincome'])])
# 정상값의 최대값으로 변환
data.loc[data['monthlyincome']>=max_value_income, 'monthlyincome'] = max_value_income

In [ ]:
max_value_income

In [ ]:
get_outlier(data['monthlyincome']).sum()

In [ ]:
data.monthlyincome.hist(bins=50)
plt.show()

# 전처리한 data파일 저장

In [ ]:
import os
os.makedirs('saved_data', exist_ok=True)
data.to_csv('data/data-v01.csv', index=False)

### 상관관계 확인
- 시각화 : heatmap

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(), #상관계수행렬
            annot=True, # 색위에 값이 나오도록 처리
            fmt='.2f', # 값의 format
            cmap='Blues', # 색 팔레트, color map
            linewidths=.5)
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
# 60-89, 90  이상 대출연체 횟수 컬럼을 제거->heatmap
sns.heatmap(data.drop(labels=['numberoftime60-89dayspastduenotworse', 'numberoftimes90dayslate'], axis=1).corr(), annot=True, fmt='.2f', cmap='Blues', linewidths=.5)
plt.show()

In [ ]:
# 파일로 저장.
data.drop(labels=['numberoftime60-89dayspastduenotworse', 'numberoftimes90dayslate'], axis=1).to_csv('data/data-v02.csv', index=False)